# Jupyter Hub-Api tutorial

---
**WARNING**

This code is auto-generated at notebook startup. Copy this file if you want to edit it.

---

## Get data from the hub-api

In [ ]:
import os
import requests

url = 'https://api.hub.coda19.com/stats/summarize?sites=115'
token = os.environ['OAUTH_TOKEN'] # Note: token has a lifespan of 4 hours. After which the notebook server must be stopped (File/Hub Control Pannel) and a login must be redone.
header = {
    "Authorization": f'Bearer {token}'
}

body = {
    "selectors": [
        {
            "resource": "Observation",
            "filters": [
                {
                    "path": "code.coding.code",
                    "operator": "is",
                    "value": "94500-6"
                },
                { 
                    "path": "interpretation.coding.display",
                    "operator": "is",
                    "value": "positive"
                },
                {
                    "path": "issued",
                    "operator": "afterOrOn",
                    "value": "2021-01-01"
                },
                {
                    "path": "issued",
                    "operator": "before",
                    "value": "2021-04-06"
                }
            ],
            "fields": [],
            "breakdown": {
                "resource": {
                    "type": "Observation",
                    "field": "issued"
                },
                "slices": {
                    "step": 86400,
                    "min": "2021-01-01",
                    "max": "2021-04-06"
                }
            }
        }
    ],
    "options": {
        "measures": {
            "continuous": [],
            "categorical": [
                "count"
            ]
        }
    }
}

# This request takes some time, so be patient before executing next cells and wait for the done output
result = requests.get(url=url, headers=header, json=body).json()
print('done')

## Visualize results

In [ ]:
!pip install pandas matplotlib # For minimal notebook only

In [ ]:
import pandas as pd
import numpy as np
from dateutil import parser

import matplotlib.pyplot as plt
from matplotlib import dates as mpl_dates

breakdowns = result[0][0]['breakdown']['result']
grouped_breakdowns = []

group_size = 9

for cur_group_index in range(0, len(breakdowns), group_size):
    group_end_index = min(cur_group_index + group_size, len(breakdowns))
    
    group_initial_period = breakdowns[cur_group_index]
    period = {
        'periodStart': parser.parse(group_initial_period['periodStart']),
        'periodCount': 0
    }
    
    grouped_breakdowns.append(period)
    
    for cur_item in range(cur_group_index, group_end_index):
        cur_period = breakdowns[cur_item]
        period['periodCount'] += cur_period['periodCount']
        

df = pd.DataFrame.from_dict(grouped_breakdowns)

# Show the data
df

In [ ]:
# Visualize the data
plt.rcParams['figure.figsize'] = [10, 5]
plt.title('Positive covid cases', fontsize='large')
plt.xlabel('Period', fontsize='large')
plt.ylabel('Covid cases', fontsize='large')

plt.plot_date(df.periodStart, df.periodCount, linestyle ='solid')
plt.tight_layout()

plt.gcf().autofmt_xdate()
date_format = mpl_dates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(date_format)
plt.show()